## 1.工程决策指南
调用 gensim.models.Word2Vec 时，参数 vector_size填多少？

- 一般工业界标准：Word2Vec 的维度通常选择在 100 到 300 之间。Google 发布的预训练模型是 300 维。
- 维度过低 (Underfitting)：无法捕捉单词之间复杂的语义关系。比如，“苹果”既是水果也是科技公司，如果维度太低，这两个维度的特征可能就被挤压混淆了。
- 维度过高 (Overfitting & Cost)：计算量大，存储变大。如果你的语料库很小（比如只有几千个句子），用 300 维会导致过拟合（每个词都独一无二，学不到通用规律）。

如果是自己从头训练：

- 小语料（< 10MB 文本）：选 50-100 维。
- 中等语料（10MB - 1GB）：选 100-200 维。
- 大语料（> 1GB）：选 200-300 维。

## 2.语义相似度的核心：余弦相似度 (Cosine Similarity)
>为什么 NLP 里常用 Cosine Similarity 而不是欧式距离 (Euclidean Distance)？

### 数学定义：给定两个向量 $A$ 和 $B$，余弦相似度计算的是它们夹角的余弦值：
$$ \text { Cosine Similarity }=\cos (\theta)=\frac{A \cdot B}{\|A\|\|B\|}=\frac{\sum_{i=1}^{n} A_{i} B_{i}}{\sqrt{\sum_{i=1}^{n} A_{i}^{2} \sqrt{\sum_{i=1}^{n} B_{i}^{2}}}}$$

>取值范围：[-1, 1]。1 表示完全方向相同（同义），0 表示正交（无关），-1 表示方向相反。

>cosine similarity 和欧式距离的核心区别：
1. 方向和距离：Cosine 关注的是方向。在向量空间中，方向代表语义。Euclidean 关注的是绝对距离。
2. 模长（Magnitude）的影响：
- 在文本中，一个词出现的频率不同，或者文档长度不同，可能导致向量模长很大。
- 比如：“很好”和“很好 很好 很好”。在 Count-based 方法中，后者模长是前者的 3 倍，欧式距离会很大，但它们语义方向是一致的。
- Cosine 通过除以模长 $\|A\|\|B\|$ 进行了归一化，消除了“长度/频率”的影响，只保留了“语义方向”。

### 3.tf-idf和word2vec的区别
|特性|TF-IDF|Word2Vec|
|:---:|:---:|:---:|
|数据表示形式	|高维、稀疏向量 (Sparse)	|低维、稠密向量 (Dense)|
|维度 (Dimensions)	|词表大小 (10k - 100k+)	|固定维度 (100 - 300)|
|语义捕捉 (Semantics)	|弱。只利用词频，无法捕捉“苹果”和“梨”相似。	|强。向量距离代表语义相似度。|
|上下文信息	|无 (Bag of Words 假设)	|有 (通过 Window context 学习)|
|OOV (未登录词) 问题	|忽略或作为新特征	|无法处理（除非用 fastText 或 UNK token）|
|可解释性	|强 (知道哪个词权重高)	|弱 (向量的每一维没有明确物理含义)|
|典型应用场景	|关键词提取、简单的垃圾邮件分类	|语义搜索、推荐系统、深度学习输入层|

### 4.实战：基于 20 Newsgroups 的 Word2Vec 文本分类
>pipeline：加载数据 -> 预处理 -> 训练W2V -> 平均池化 -> 分类 -> 评估。

In [1]:
import numpy as np
import re
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from gensim.models import Word2Vec
import time

# ==========================================
# 1. 加载真实数据集 (只选4个类别以节省时间，模拟实际业务中的多分类)
# ==========================================
print("正在加载 20 Newsgroups 数据集...")
categories = ['rec.autos', 'sci.space', 'comp.graphics', 'talk.politics.mideast']
# remove参数用于去除新闻组数据的元数据（标题、页脚等），防止模型“作弊”
newsgroups = fetch_20newsgroups(subset='all', categories=categories, 
                                remove=('headers', 'footers', 'quotes'))

X_text = newsgroups.data  # 原始文本列表
y_labels = newsgroups.target # 标签
target_names = newsgroups.target_names

print(f"数据加载完毕。样本数: {len(X_text)}, 类别: {target_names}")
print("-" * 30)

# ==========================================
# 2. 文本预处理 (Engineering Vital Step)
# ==========================================
# Word2Vec 需要的输入是：List[List[str]]，即分好词的句子列表
def simple_preprocess(text):
    # 1. 转小写
    text = text.lower()
    # 2. 去除特殊符号，只保留字母和空格 (简单的正则)
    text = re.sub(r'[^a-z\s]', '', text)
    # 3. 分词 (split by space)
    tokens = text.split()
    # 4. 去除过短的停用词 (简单过滤长度<3的词，如 is, a, to)
    tokens = [t for t in tokens if len(t) > 2]
    return tokens

print("正在预处理文本...")
start_time = time.time()
# 处理所有文档
tokenized_corpus = [simple_preprocess(doc) for doc in X_text]
print(f"预处理耗时: {time.time() - start_time:.2f}s")

# 预览一下处理后的第一条数据
print(f"样例数据 (前10个词): {tokenized_corpus[0][:10]}")
print("-" * 30)

# ==========================================
# 3. 训练 Word2Vec 模型
# ==========================================
print("正在训练 Word2Vec (这可能需要几秒钟)...")
# vector_size=100: 工业界常用起步维度
# window=5: 上下文窗口
# min_count=5: 过滤掉出现次数少于5次的低频词 (非常重要！减少噪音)
w2v_model = Word2Vec(sentences=tokenized_corpus, 
                     vector_size=100, 
                     window=5, 
                     min_count=5, 
                     workers=4)

print("Word2Vec 训练完毕。")
# 检查一下模型学到了什么
check_word = 'car'
if check_word in w2v_model.wv:
    print(f"与 '{check_word}' 最相似的词: {w2v_model.wv.most_similar(check_word, topn=3)}")
print("-" * 30)

# ==========================================
# 4. 特征工程：Mean Pooling (词向量 -> 句向量)
# ==========================================
def get_document_vector(doc_tokens, model):
    vectors = []
    for word in doc_tokens:
        if word in model.wv:
            vectors.append(model.wv[word])
    
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        # 如果整个文档的词都不在词表里（极端情况），返回零向量
        return np.zeros(model.vector_size)

print("正在将文档转换为向量矩阵...")
X_vectors = np.array([get_document_vector(doc, w2v_model) for doc in tokenized_corpus])

print(f"特征矩阵形状: {X_vectors.shape}") # 应该是 (样本数, 100)

# ==========================================
# 5. 模型训练与评估
# ==========================================
# 拆分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_vectors, y_labels, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000) # 增加迭代次数以保证收敛
print("正在训练分类器 (Logistic Regression)...")
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("-" * 30)
print("【分类报告】")
print(classification_report(y_test, y_pred, target_names=target_names))

正在加载 20 Newsgroups 数据集...
数据加载完毕。样本数: 3890, 类别: ['comp.graphics', 'rec.autos', 'sci.space', 'talk.politics.mideast']
------------------------------
正在预处理文本...
预处理耗时: 0.10s
样例数据 (前10个词): ['believe', 'many', 'people', 'will', 'happy', 'have', 'this', 'information', 'please', 'post']
------------------------------
正在训练 Word2Vec (这可能需要几秒钟)...
Word2Vec 训练完毕。
与 'car' 最相似的词: [('time', 0.9000201225280762), ('something', 0.89157634973526), ('thing', 0.8861873149871826)]
------------------------------
正在将文档转换为向量矩阵...
特征矩阵形状: (3890, 100)
正在训练分类器 (Logistic Regression)...
------------------------------
【分类报告】
                       precision    recall  f1-score   support

        comp.graphics       0.75      0.70      0.72       186
            rec.autos       0.67      0.75      0.71       221
            sci.space       0.72      0.62      0.66       206
talk.politics.mideast       0.77      0.84      0.80       165

             accuracy                           0.72       778
            macr

### 1.预处理的必要性 (min_count)
将min_count设为5。在真实语料中，存在大量只出现 1-2 次的词（拼写错误、生僻专有名词）。这些词不仅学不出好的向量（样本太少），还会由噪声干扰模型的权重更新，增加内存开销。过滤掉它们能显著提升模型鲁棒性。

### 2.OOV (Out of Vocabulary) 处理
在 get_document_vector 函数中，我加了一个判断：if word in model.wv。

- 工程现实：测试集里的词，可能在训练集中从未出现过。

- 如果不加判断：程序会直接报错（KeyError）。

- 如果列表为空：必须返回一个全 0 向量作为兜底，保证矩阵维度一致。